In [1]:
import pandas as pd
import numpy as np
import math

In [ ]:
# Define maximum and minimum
max_distance = 10.0
min_distance = 0.1

# Read public services data
PUBLIC_SERVICES_WEIGHTS = pd.read_csv("data/public_services_weights.csv")
public_services_data = pd.read_csv("data/public_services.csv")

# Clean data
public_services_data["main_cat"] = public_services_data["main_cat"] .apply(lambda x: x.strip().title())
public_services_data["sub_cat"] = public_services_data["sub_cat"] .apply(lambda x: x.strip().title())
# Filter out irrelevant row
public_services_data = public_services_data[(public_services_data["sub_cat"]!="Delete")&(public_services_data["sub_cat"]!="Delete")]

# Read listing data
real_estate_listings = pd.read_csv("data/riyadh_sample.csv")
real_estate_listings = real_estate_listings
real_estate_listings["rs_score"] = np.nan

In [2]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Radius of the Earth in kilometers
    radius = 6371.0

    # Calculate the differences between the latitudes and longitudes
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Apply the Haversine formula
    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = radius * c

    return distance

def calculate_proximity_score(distance):
    """
    Calculates a promixity score for a given distance, on a scale of minimum to maximum
    The maximum and minimum distance should be defined in kilometers
    """

    if distance > max_distance:  # Disance is too far
        proximity_score = 0

    elif distance < min_distance:  # Distance is too close
        proximity_score = 0

    else:
        # Assign a score based on where the distance lies within the maximum and minimum
        proximity_score = 1 - (
            (distance - min_distance) / (max_distance - min_distance)
        )

    return proximity_score


In [4]:
for i, listing_row in real_estate_listings.iterrows():
    
    listings_lat = listing_row['lat']
    listings_long = listing_row['lng']

    # Get distance of each public service from the listing
    public_services_data["distance"] = public_services_data.apply(
        lambda row: calculate_distance(
            lat1=listings_lat,
            lon1=listings_long,
            lat2=row["lat"],
            lon2=row["lng"],
        ),
        axis=1,
    )


    # Determine the proximity score for the each distances
    public_services_data["proximity_score"] = public_services_data["distance"].apply(
        lambda x: calculate_proximity_score(x)
    )

    # Keep the maximum proximity score in each sub category
    proximity_scores = (
        public_services_data.groupby(["main_cat", "sub_cat"])["proximity_score"]
        .max()
        .reset_index()
    )

    # Get the weighted score
    proximity_scores = pd.merge(
        proximity_scores,
        PUBLIC_SERVICES_WEIGHTS,
        how="left",
        on=["main_cat", "sub_cat"],
    )

    proximity_scores["weighted_score"] = (
        proximity_scores["proximity_score"] * proximity_scores["weight"]
    )

    # Get the scores for the various groups
    category_scores = (
        proximity_scores.groupby("main_cat")["weighted_score"].sum().reset_index()
    )

    # Find the average category score (final rs_score)
    rs_score = round(category_scores["weighted_score"].mean(), 4)

    real_estate_listings.loc[i, "rs_score"] = rs_score


In [5]:
real_estate_listings

,l_neighborhood,deal_price,area_size,meter_price,lng,lat,uuid,rs_score
0,rd,1080000,350.00,3086,46.757046,24.727277,c1442e7b-ddd2-48db-b7c4-79d49412754e,0.2132
1,ml,11000000,560.00,19643,46.613572,24.801113,eaab2273-b0da-407f-a0e2-170b94f19d9d,0.7886
2,ml,4590000,600.00,7650,46.594482,24.794327,2be6d20a-bf6d-4a32-8459-d80deb18979a,0.7518
3,ml,41760000,2610.00,16000,46.627284,24.799022,1f0a475e-4d34-42ce-af59-de8cebe00664,0.7992
4,nr,2000000,500.00,4000,46.656898,24.885271,4fc312b5-bbe7-4f45-9c22-25a5ad94ae79,0.6514
...,...,...,...,...,...,...,...,...
582,shf,1226000,875.00,1401,46.707118,24.566447,85f45195-32e5-4335-b4a6-4896dc1c343d,0.7270
583,rd,1500000,735.00,2041,46.758873,24.727120,78a0cfe3-fc86-47fa-8f80-05584ce34450,0.2037
584,ml,9300000,925.00,10054,46.625785,24.793738,9f24a6e8-31be-4b06-9796-9ce2d9b364ac,0.7950
585,nr,1493640,324.00,4610,46.656574,24.848594,30b36baa-abc8-43d6-8940-b0bf8c4fc601,0.7628


In [6]:
real_estate_listings.to_csv("results.csv", index=False)